### Get TM1 Data

In [ ]:
from TM1py.Services import TM1Service
from TM1py.Utils.Utils import element_names_from_element_unique_names, build_pandas_dataframe_from_cellset
from creds import tm1_credentials

In [ ]:
with TM1Service(address=tm1_credentials["address"], port=tm1_credentials["port"], user=tm1_credentials["user"], password=tm1_credentials["password"], ssl=tm1_credentials["ssl"]) as tm1:
    data = tm1.cubes.cells.execute_view(cube_name='OPS Sales', view_name='Optimization View', private=False)
    df = build_pandas_dataframe_from_cellset(data, multiindex=False)

In [ ]:
df = df.drop(['Version', 'OPS Sales Source', 'OPS Sales Measure'], axis =1)

In [ ]:
df = df.set_index(['Store', 'Product', 'Time Date'])

In [ ]:
df = df.fillna(0)

In [ ]:
df.head()

### Import Pulp

In [ ]:
import pulp

### Define marketing spend variables

In [ ]:
marketingspend = pulp.LpVariable.dicts("marketingspend", ((store, product, day) for store, product, day in df.index.unique()),
                                     lowBound=0,
                                     cat='Integer')

In [ ]:
len(marketingspend)

In [ ]:
len([(store, product, day) for store, product, day in df.index.unique()])

### Setup problem

In [ ]:
model = pulp.LpProblem("Profit maximising problem", pulp.LpMaximize)

In [ ]:
model

In [ ]:
model += pulp.lpSum([df[df['Account']=='4999'].loc[(store, product, day)]['Values']*df[df['Account']=='4994'].loc[(store, product, day)]['Values'] for store, product, day in df.index.unique()] +
           [(marketingspend[(store, product, day)]*df[df['Account']=='4996'].loc[(store, product, day)]['Values'])*df[df['Account']=='4994'].loc[(store, product, day)]['Values'] for store, product, day in df.index.unique() ])
           

### Add Constraints

In [ ]:
for store, product, day in df.index.unique():
    model += df[df['Account']=='4999'].loc[(store, product, day)]['Values'] + marketingspend[(store, product, day)]*df[df['Account']=='4996'].loc[(store, product, day)]['Values'] <= df[df['Account']=='4998'].loc[(store, product, day)]['Values']

In [ ]:
model += pulp.lpSum(marketingspend) <= 33381  

### Solve Problem

In [ ]:
model.solve()
print(pulp.LpStatus[model.status])
print(pulp.value(model.objective))

In [ ]:
cellset = {(day, 'Optimized Actual', '4997', product, store, 'Base', 'Amount'):marketingspend[(store, product, day)].varValue for store, product, day in df.index.unique()}

In [ ]:
cellset

In [ ]:
with TM1Service(address=tm1_credentials["address"], port=tm1_credentials["port"], user=tm1_credentials["user"], password=tm1_credentials["password"], ssl=tm1_credentials["ssl"]) as tm1:
    tm1.cubes.cells.write_values('OPS Sales', cellset, dimensions=['Time Date', 'Version', 'Account', 'Product', 'Store', 'OPS Sales Source', 'OPS Sales Measure'])